# 📊 Analyse Exploratoire des Données (EDA)
## Santander Customer Transaction Prediction

---

### Objectifs de ce notebook :
1. Charger et comprendre le dataset
2. Analyser la distribution de la variable cible
3. Explorer les features
4. Identifier les patterns et corrélations
5. Détecter les valeurs aberrantes
6. Visualiser les données

In [ ]:
# Import des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Affichage de toutes les colonnes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Bibliothèques importées avec succès")

## 1️⃣ Chargement des données

In [ ]:
# Charger les données
print("📥 Chargement des données...")

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

print(f"✅ Données chargées !")
print(f"   - Train set : {train.shape[0]:,} lignes x {train.shape[1]} colonnes")
print(f"   - Test set  : {test.shape[0]:,} lignes x {test.shape[1]} colonnes")

In [ ]:
# Aperçu des premières lignes
print("📊 Aperçu du dataset d'entraînement :")
train.head()

In [ ]:
# Informations générales
print("ℹ️ Informations générales :")
train.info()

In [ ]:
# Statistiques descriptives
print("📈 Statistiques descriptives :")
train.describe()

## 2️⃣ Analyse de la variable cible

In [ ]:
# Distribution de la variable cible
target_counts = train['target'].value_counts()
target_pct = train['target'].value_counts(normalize=True) * 100

print("🎯 Distribution de la variable cible :")
print(f"   - Classe 0 (Pas de transaction) : {target_counts[0]:,} ({target_pct[0]:.2f}%)")
print(f"   - Classe 1 (Transaction)        : {target_counts[1]:,} ({target_pct[1]:.2f}%)")
print(f"\n⚠️ Déséquilibre : {target_pct[0] / target_pct[1]:.2f}:1")

In [ ]:
# Visualisation de la distribution de la cible
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Barplot
target_counts.plot(kind='bar', ax=axes[0], color=['#3498db', '#e74c3c'])
axes[0].set_title('Distribution de la Variable Cible', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Classe')
axes[0].set_ylabel('Nombre d\'observations')
axes[0].set_xticklabels(['Pas de transaction', 'Transaction'], rotation=0)

# Pie chart
axes[1].pie(
    target_counts,
    labels=['Pas de transaction', 'Transaction'],
    autopct='%1.1f%%',
    colors=['#3498db', '#e74c3c'],
    startangle=90
)
axes[1].set_title('Proportion des Classes', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 3️⃣ Analyse des features

In [ ]:
# Vérifier les valeurs manquantes
missing = train.isnull().sum()
missing_pct = (missing / len(train)) * 100

if missing.sum() == 0:
    print("✅ Aucune valeur manquante dans le dataset !")
else:
    print("⚠️ Valeurs manquantes détectées :")
    print(missing[missing > 0])

In [ ]:
# Sélectionner uniquement les features (var_0 à var_199)
feature_cols = [col for col in train.columns if col.startswith('var_')]
print(f"📊 Nombre de features : {len(feature_cols)}")

# Statistiques sur les features
features_stats = train[feature_cols].describe().T
features_stats['range'] = features_stats['max'] - features_stats['min']
features_stats.head(10)

In [ ]:
# Distribution de quelques features
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

# Sélectionner 6 features aléatoires
random_features = np.random.choice(feature_cols, 6, replace=False)

for i, feature in enumerate(random_features):
    axes[i].hist(train[feature], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
    axes[i].set_title(f'Distribution de {feature}', fontweight='bold')
    axes[i].set_xlabel('Valeur')
    axes[i].set_ylabel('Fréquence')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4️⃣ Corrélations

In [ ]:
# Corrélation des features avec la cible
correlations = train[feature_cols].corrwith(train['target']).sort_values(ascending=False)

print("🔗 Top 10 features les plus corrélées (positivement) avec la cible :")
print(correlations.head(10))

print("\n🔗 Top 10 features les plus corrélées (négativement) avec la cible :")
print(correlations.tail(10))

In [ ]:
# Visualisation des corrélations
fig, ax = plt.subplots(figsize=(12, 6))

# Top 20 corrélations absolues
top_corr = correlations.abs().sort_values(ascending=False).head(20)
colors = ['red' if correlations[idx] < 0 else 'green' for idx in top_corr.index]

top_corr.plot(kind='barh', ax=ax, color=colors, alpha=0.7)
ax.set_title('Top 20 Features par Corrélation avec la Cible', fontsize=14, fontweight='bold')
ax.set_xlabel('Corrélation Absolue')
ax.set_ylabel('Feature')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5️⃣ Analyse par classe

In [ ]:
# Comparer les distributions pour les features les plus corrélées
top_features = correlations.abs().sort_values(ascending=False).head(6).index

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, feature in enumerate(top_features):
    # Distribution par classe
    train[train['target'] == 0][feature].hist(
        bins=50, alpha=0.5, label='Pas de transaction',
        ax=axes[i], color='blue'
    )
    train[train['target'] == 1][feature].hist(
        bins=50, alpha=0.5, label='Transaction',
        ax=axes[i], color='red'
    )
    
    axes[i].set_title(f'{feature} (corr={correlations[feature]:.3f})', fontweight='bold')
    axes[i].set_xlabel('Valeur')
    axes[i].set_ylabel('Fréquence')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6️⃣ Détection des valeurs aberrantes

In [ ]:
# Boxplot pour quelques features
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

sample_features = np.random.choice(feature_cols, 6, replace=False)

for i, feature in enumerate(sample_features):
    train.boxplot(column=feature, by='target', ax=axes[i])
    axes[i].set_title(f'Boxplot de {feature}')
    axes[i].set_xlabel('Target')
    axes[i].set_ylabel('Valeur')

plt.suptitle('')  # Supprimer le titre automatique
plt.tight_layout()
plt.show()

## 📝 Conclusions de l'EDA

### Points clés :
1. **Dataset déséquilibré** : ~90% classe 0, ~10% classe 1
2. **Pas de valeurs manquantes** : Dataset propre
3. **200 features anonymisées** : Difficile d'interpréter le sens métier
4. **Corrélations faibles** : Les features ont des corrélations relativement faibles avec la cible
5. **Présence d'outliers** : Certaines features ont des valeurs aberrantes

### Prochaines étapes :
1. Preprocessing et feature engineering
2. Gestion du déséquilibre des classes
3. Feature selection
4. Entraînement de modèles

In [ ]:
# Sauvegarder les insights
insights = {
    'n_samples': len(train),
    'n_features': len(feature_cols),
    'class_distribution': target_counts.to_dict(),
    'class_imbalance_ratio': float(target_pct[0] / target_pct[1]),
    'missing_values': int(missing.sum()),
    'top_correlated_features': correlations.abs().sort_values(ascending=False).head(10).to_dict()
}

print("✅ Analyse exploratoire terminée !")
print(f"\n📊 Insights sauvegardés :")
for key, value in insights.items():
    print(f"   - {key}: {value}")